In [1]:
import numpy as np
import pandas as pd
import glob
import torch
import pickle
import os
import glob, sys

os.environ['KMP_DUPLICATE_LIB_OK']='True'
sys.path.append(os.path.join(os.path.abspath(os.getcwd()),".."))
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))


In [25]:
# MSEs for FERL vs random per feature per number of trace
gen_data_file = "FERL_traces_{}_gen_data.p"
table = parent_dir + "/data/FERL_gen/" + gen_data_file.format("table")
laptop = parent_dir + "/data/FERL_gen/" + gen_data_file.format("laptop")
coffee = parent_dir + "/data/FERL_gen/" + gen_data_file.format("coffee")
laptopmoving = parent_dir + "/data/FERL_gen/" + gen_data_file.format("laptopmoving")
proxemics = parent_dir + "/data/FERL_gen/" + gen_data_file.format("proxemics")
betweenobjects = parent_dir + "/data/FERL_gen/" + gen_data_file.format("betweenobjects")

datafiles = [table, laptop, laptopmoving, coffee, proxemics, betweenobjects]
titles = ['table', 'laptop', 'test laptop','coffee', 'proxemics', 'between objects']
H1_table = []
H2_table = []
H3_table = []
for i, dfile in enumerate(datafiles):
    if titles[i] in ["table", "laptop", "coffee", "proxemics"]:
        rel_data = 1
    elif titles[i] in ["test laptop"]:
        rel_data = 2
    elif titles[i] in ["between objects"]:
        rel_data = 3

    with open(dfile, 'rb') as f:
        df = pickle.load(f, encoding='latin1')
        
        # Table for H1.
        trained_df = df[df["config/n_train_trajs"] == 10*rel_data]
        for index, row in trained_df.iterrows():
            H1_table.append([titles[i]] + ["FERL"] + [np.mean(np.array(row["delta_dist"])**2)])
            H1_table.append([titles[i]] + ["random"] + [np.mean(np.array(row["delta_random_dist"])**2)])

        # Table for H2.
        clean_df = df[df["config/n_train_trajs"] > rel_data]
        for index, row in clean_df.iterrows():         
            H2_table.append([titles[i]] + [row["config/n_train_trajs"] / rel_data] + [np.mean(np.array(row["delta_dist"])**2)])

        # Table for H3.
        list_n_traj = clean_df['config/n_train_trajs'].unique()
        list_n_traj.sort()

        for j in list_n_traj:
            temp = [experiment for experiment in df[df['config/n_train_trajs'] == j]['delta_dist']]
            MSEs = np.array([np.mean(x**2) for x in temp])
            H3_table.append([titles[i]] + [j / rel_data] + [MSEs.std()/np.sqrt(len(MSEs))])
                      
df = pd.DataFrame(H1_table, columns=['feature', 'method', 'MSE'])
df.to_csv(parent_dir + '/data/CSV/H1_FERL.csv', index=False)
                      
df = pd.DataFrame(H2_table, columns=['feature', 'relative_data', 'MSE'])
df.to_csv(parent_dir + '/data/CSV/H2_FERL.csv', index=False)
                      
df = pd.DataFrame(H3_table, columns=['feature', 'relative_data', 'MSE_SE'])
df.to_csv(parent_dir + '/data/CSV/H3_FERL.csv', index=False)

In [3]:
dataframes

[    config/n_train_trajs                                         delta_dist  \
 0                      8  [0.05109330087243258, 0.030832422431065654, -0...   
 1                      9  [0.05523034005700289, 0.003929597790791606, -0...   
 2                      9  [-0.1622357758945352, 0.25286520784671507, 0.1...   
 3                      7  [-0.07886044115485014, 0.0506946570235034, 0.0...   
 4                      6  [-0.003063300556553017, 0.10676540201480589, 0...   
 ..                   ...                                                ...   
 85                     7  [0.049040516906368126, 0.01674161618526182, -0...   
 86                     3  [0.028552135520564903, -0.056700783316538717, ...   
 87                     2  [0.14229275851784884, -0.09645776683513918, -0...   
 88                     7  [0.14465012221871554, 0.23948807781512937, 0.1...   
 89                     8  [-0.15254954428137601, 0.046902609522893046, 0...   
 
                                     d

In [13]:
# Cost MSEs for FERL vs MEIRL per case per number of traces/demonstrations
def get_MSEs(df):
    # create vector of all n_trajs
    if "config/n_train_trajs" in df:
        key = "config/n_train_trajs"
    else:
        key = "n_traj"
    list_n_traj = df[key].unique()
    list_n_traj.sort()
    
    # create a list of delta_dist arrays
    MSEs = []
    for i in list_n_traj:
        temp = [experiment for experiment in df[df[key] == i]['delta_dist']]
        MSE = [np.mean(x**2) for x in temp]
        MSEs.append(MSE)
    return MSEs

FERLfiles = ["FERL_cost_laptop.p", "FERL_cost_table.p", "FERL_cost_proxemics.p"]
MEIRLfiles = ["MEIRLcost_tablelaptop_case1.p", "MEIRLcost_tablelaptop_case2.p", "MEIRLcost_tableproxemics_case3.p"]
FERLdfs = []
MEIRLdfs = []
for i in range(len(FERLfiles)):
    FERLfile = parent_dir + "/data/FERL_gen/" + FERLfiles[i]
    MEIRLfile = parent_dir + "/data/FERL_gen/" + MEIRLfiles[i]
    with open(FERLfile, 'rb') as f:
        FERLdfs.append(pickle.load(f,encoding="latin1"))
    with open(MEIRLfile, 'rb') as m:
        MEIRLdfs.append(pickle.load(m,encoding="latin1"))
        
FERL1files = ["FERL1_traces_laptop_gen_data.p", "FERL1_traces_table_gen_data.p", "FERL1_traces_proxemics_gen_data.p"]
FERL1dfs = []
for i in range(len(FERL1files)):
    FERLfile = parent_dir + "/data/FERL_gen/" + FERL1files[i]
    with open(FERLfile, 'rb') as f:
        FERL1dfs.append(pickle.load(f,encoding="latin1"))
        
FERLMSEs = []
MEIRLMSEs = []
for case in range(len(FERLfiles)):
    MSEs11 = get_MSEs(FERL1dfs[case])
    MSEs1 = get_MSEs(FERLdfs[case])
    MSEs2 = get_MSEs(MEIRLdfs[case])
    FERLMSEs.append(np.vstack((np.array(MSEs11), np.array(MSEs1))).tolist())
    MEIRLMSEs.append(MSEs2)

In [16]:
H4_table = []
H5_table = []
for case in range(3):
    for num_trajs in range(10):
        for seed in range(10):
            H4_table.append(["case{}".format(case+1)] + [num_trajs+1] + ["FERL expert"] + [FERLMSEs[case][num_trajs][seed]])
            H4_table.append(["case{}".format(case+1)] + [num_trajs+1] + ["MEIRL expert"] + [MEIRLMSEs[case][num_trajs][seed]])
        if num_trajs == 9:
            tmp_mse = FERLMSEs[case][num_trajs]
            H5_table.append(["case{}".format(case+1)] + ["FERL expert"] + [np.array(tmp_mse).std()/np.sqrt(len(tmp_mse))])
            tmp_mse = MEIRLMSEs[case][num_trajs]
            H5_table.append(["case{}".format(case+1)] + ["MEIRL expert"] + [np.array(tmp_mse).std()/np.sqrt(len(tmp_mse))])
df = pd.DataFrame(H4_table, columns=['task', 'num_data', 'method', 'MSE'])
df.to_csv(parent_dir + '/data/CSV/H4_reward.csv', index=False)
df = pd.DataFrame(H5_table, columns=['task', 'method', 'MSE_SE'])
df.to_csv(parent_dir + '/data/CSV/H5_reward.csv', index=False)

In [24]:
# Induced Costs FERL vs MEIRL per case
files = ["InducedCosts_case1.p", "InducedCosts_case2.p", "InducedCosts_case3.p"]
GT_costs = []
FERL_costs = []
MEIRL_costs = []
H4_table_induced = []
for i in range(len(files)):
    costfile = parent_dir + "/data/FERL_gen/" + files[i]
    with open(costfile, 'rb') as f:
        costs = pickle.load(f, encoding="latin1")
        MEIRL_costs.append(costs[1])
        FERL_costs.append(costs[2])
        for j in range(len(costs[1])):
            H4_table_induced.append([j] + ["case{}".format(i+1)] + ["FERL expert"] + [costs[2][j]])
            H4_table_induced.append([j] + ["case{}".format(i+1)] + ["MEIRL expert"] + [costs[1][j]])
df = pd.DataFrame(H4_table_induced, columns=['ID', 'task', 'method', 'MSE'])
df.to_csv(parent_dir + '/data/CSV/H4_induced.csv', index=False)